# Setup

In [6]:
%pip install -qU IProgress
%pip install -qU ipywidgets
%pip install -qU python-dotenv
%pip install -qU tqdm

%pip install -qU langchain
%pip install -qU langchain-community
%pip install -qU langchain-openai 
%pip install -qU langchain-qdrant 

%pip install -qU pymupdf
%pip install -qU ragas==0.1.20

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.19 requires langchain<0.3.0,>=0.2.17, but you have langchain 0.3.14 which is incompatible.
langchain-community 0.2.19 requires langchain-core<0.3.0,>=0.2.43, but you have langchain-core 0.3.29 which is incompatible.
langchain-openai 0.1.25 requires langchain-core<0.3.0,>=0.2.40, but you have langchain-core 0.3.29 which is incompatible.
ragas 0.1.20 requires langchain-core<0.3, but you have langchain-core 0.3.29 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ragas 0.1.20 requires langchain-core<0.3, but you have langchain-core 0.3.29 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ragas 0.1.20 requires langchain-core<0.3, but you have langchain-core 0.3.29 which is incompatible.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Environment Variables

In [7]:
# Get environment variables

import os
import uuid

from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()

True

# Utilities

In [8]:
from langchain_community.document_loaders import DirectoryLoader 
from langchain_community.document_loaders import PyMuPDFLoader

def process_directory(path: str, glob: str, loader_cls: str, use_multithreading=True):
	
	loader = DirectoryLoader(path=path, glob=glob, show_progress=True, loader_cls=loader_cls, use_multithreading=use_multithreading)
	
	docs = loader.load()
	
	return docs


def test_process_directory():
	docs = process_directory('docs/', '**/*.pdf', PyMuPDFLoader, True)
	print(len(docs))

test_process_directory()

100%|██████████| 2/2 [00:00<00:00,  2.22it/s]

334


In [9]:
# Create embeddings using OpenAI

from langchain_openai import OpenAIEmbeddings

def create_embeddings_openai(model="text-embedding-ada-002") -> OpenAIEmbeddings:

	# Initialize the OpenAIEmbeddings class
	embeddings = OpenAIEmbeddings(model=model)

	return embeddings

#####

def test_create_embeddings_openai():
	text = "What is the annual revenue of Uber?"
	
	embeddings = create_embeddings_openai()
	vector = embeddings.embed_query(text)
	
	print(vector)

test_create_embeddings_openai()

[0.020656345412135124, -0.04052777588367462, 0.02295876294374466, -0.02543124556541443, -0.010001124814152718, -0.023979153484106064, -0.021938374266028404, -0.01661403477191925, -0.02121886797249317, -0.02864939719438553, 0.02060401812195778, 0.007528643123805523, -0.007450151722878218, -0.013232358731329441, 0.0011029690504074097, -0.0037643215619027615, 0.012774491682648659, -0.010445910505950451, 0.017660588026046753, 0.015724463388323784, -0.0030971437226980925, -0.0004223006253596395, -0.04633614793419838, 0.006590015720576048, 0.0023890850134193897, 0.029198838397860527, 0.024685576558113098, 0.0007219582912512124, 0.025025706738233566, -0.02121886797249317, 0.008274312131106853, 0.004097910597920418, -0.025391999632120132, 0.02114037796854973, -0.036184582859277725, -0.0036825595889240503, 0.008235066197812557, 0.0011201391462236643, 0.03835618123412132, 0.02650396339595318, 0.01725504733622074, 0.0068156784400343895, -0.009726405143737793, -0.005200061947107315, -0.01828851923

In [10]:
# Create a text splitter using recursive character text splitter

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/

from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_docs_recursive(docs: list, chunk_size=500, chunk_overlap=50) -> list:

	text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

	chunks = text_splitter.split_documents(docs)

	return chunks

#####

def test_chunk_docs_recursive(): 
	docs = process_directory("docs/", "**/*.pdf", PyMuPDFLoader, True)
	chunks = chunk_docs_recursive(docs=docs)
	
	print(f"\nNumber of chunks = {len(chunks)}\n")
	print(f"First chunk = {chunks[0].page_content}")

test_chunk_docs_recursive()

100%|██████████| 2/2 [00:00<00:00,  2.23it/s]


Number of chunks = 3398

First chunk = UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
 
(Mark One)
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended December 31, 2023
OR
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 FOR THE
TRANSITION PERIOD FROM                      TO
Commission File Number 001-38846
Lyft, Inc.
(Exact name of registrant as specified in its charter)
Delaware
20-8809830


In [11]:
# Create a Qdrant vector store

from langchain_core.embeddings import Embeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

def create_qdrant_vector_store(location: str, 
							   collection_name: str, 
							   vector_size: int, 
							   embeddings: Embeddings, 
							   docs: list) -> QdrantVectorStore:

	# Initialize the Qdrant client
	qdrant_client = QdrantClient(location=location)

	# Create a collection in Qdrant
	qdrant_client.create_collection(collection_name=collection_name, 
								 vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE))

	# Initialize QdrantVectorStore with the Qdrant client
	qdrant_vector_store = QdrantVectorStore(client=qdrant_client, 
										 collection_name=collection_name, embedding=embeddings)
	
	# Add the docs to the vector store
	qdrant_vector_store.add_documents(docs)
	
	return qdrant_vector_store

#####

def test_create_qdrant_vector_store():
	docs = process_directory("docs/", "**/*.pdf", PyMuPDFLoader, True)
	chunks = chunk_docs_recursive(docs=docs)
	
	embeddings = create_embeddings_openai()
	vector_store = create_qdrant_vector_store(":memory:", "holiday-test", 1536, embeddings, chunks)
	
	print(vector_store.collection_name)

test_create_qdrant_vector_store()

100%|██████████| 2/2 [00:00<00:00,  2.20it/s]


holiday-test


In [12]:
# Create a Qdrant retriever

from langchain_core.retrievers import BaseRetriever
from langchain_qdrant import QdrantVectorStore

def create_retriever_qdrant(vector_store: QdrantVectorStore) -> BaseRetriever:

	retriever = vector_store.as_retriever()

	return retriever

#####

def test_create_retriever_qdrant(text):
	docs = process_directory("docs/", "**/*.pdf", PyMuPDFLoader, True)
	chunks = chunk_docs_recursive(docs=docs)
	
	embeddings = create_embeddings_openai()
	vector_store = create_qdrant_vector_store(":memory:", "holiday-test", 1536, embeddings, chunks)
	
	retriever = create_retriever_qdrant(vector_store)
	
	docs = retriever.invoke(text)	
	print(docs[0])

print("\nTESTING QDRANT")
test_create_retriever_qdrant("What is the annual revenue for Uber?")


TESTING QDRANT


100%|██████████| 2/2 [00:00<00:00,  2.03it/s]


page_content='UBER TECHNOLOGIES, INC.
CONSOLIDATED STATEMENTS OF OPERATIONS
(In millions, except share amounts which are reflected in thousands, and per share amounts)
Year Ended December 31,
2021
2022
2023
Revenue
$
17,455 
$
31,877 
$
37,281 
Costs and expenses
Cost of revenue, exclusive of depreciation and amortization shown separately below
9,351 
19,659 
22,457 
Operations and support
1,877 
2,413 
2,689 
Sales and marketing
4,789 
4,756 
4,356 
Research and development
2,054 
2,798 
3,164' metadata={'source': 'docs\\uber.pdf', 'file_path': 'docs\\uber.pdf', 'page': 76, 'total_pages': 191, 'format': 'PDF 1.4', 'title': '0001543151-24-000012', 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group', 'subject': 'Form 10-K filed on 2024-02-15 for the period ending 2023-12-31', 'keywords': '0001543151-24-000012; ; 10-K', 'creator': 'EDGAR Filing HTML Converter', 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creationDate': "D:20240215163245-05'00'", 'modDate': "D:2024021

In [13]:
# Create a prompt template

# https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/#chatprompttemplate
# https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

from langchain.prompts import ChatPromptTemplate

def create_chat_prompt_template(template: str = None) -> ChatPromptTemplate:
	
	if template is None:
		template = '''
		You are an expert assistant designed to help users analyze and answer questions about 10K annual reports filed by publicly traded companies. Users may ask about specific sections, financial metrics, trends, or comparisons across multiple reports. Your role is to provide accurate, concise, and relevant answers, referencing appropriate sections or data points where applicable.

		When responding, adhere to the following principles:

		Understand the Question: Identify whether the query is focused on a specific company, year, or metric, or if it spans multiple reports for comparison.
		Clarify Uncertainty: If a user's question is unclear, ask for clarification or additional context.
		Locate and Reference Information: Use relevant sections of the 10K report(s), such as MD&A, Financial Statements, Risk Factors, or Notes to Financial Statements, to back up your answers.
		Synthesize Data: Provide summaries or insights when the question involves comparing data or trends across multiple reports.
		Stay Objective: Avoid providing subjective opinions or interpretations beyond the factual content in the reports.
		Example User Queries and Expected Responses:

		"What was the revenue for Company X in 2022 and 2023?"

		Locate and report the revenue figures from the Income Statements of the respective 10K reports for 2022 and 2023.
		"What are the main risk factors for Company Y in its latest report?"

		Summarize the key risk factors from the most recent 10K report's "Risk Factors" section.
		"How did the operating income of Company Z change over the last three years?"

		Extract operating income figures from the 10K reports for the past three years and provide a brief comparison.
		"Compare the debt levels of Company A and Company B in 2023."

		Retrieve debt-related figures from the Balance Sheets or Notes to Financial Statements of both companies and summarize the comparison.
		"What trends are evident in Company W's R&D expenses over the last five years?"

		Summarize trends using data from the Income Statements or footnotes for R&D expenses across five consecutive 10K reports.
		Assumptions and Constraints:

		Only use inforomation from 10K reports provided in the context below. 
		For complex queries spanning multiple reports, provide a structured summary highlighting key comparisons or trends.
		If certain information is unavailable, state so clearly and suggest alternative approaches to obtain it.

		Now it's your turn!
		
		{question}

		{context}
		'''
	
	prompt = ChatPromptTemplate.from_template(template)

	return prompt

#####

def test_create_chat_prompt_template():
	prompt = create_chat_prompt_template()
	
	print(prompt)

test_create_chat_prompt_template()

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\n\t\tYou are an expert assistant designed to help users analyze and answer questions about 10K annual reports filed by publicly traded companies. Users may ask about specific sections, financial metrics, trends, or comparisons across multiple reports. Your role is to provide accurate, concise, and relevant answers, referencing appropriate sections or data points where applicable.\n\n\t\tWhen responding, adhere to the following principles:\n\n\t\tUnderstand the Question: Identify whether the query is focused on a specific company, year, or metric, or if it spans multiple reports for comparison.\n\t\tClarify Uncertainty: If a user\'s question is unclear, ask for clarification or additional context.\n\t\tLocate and Reference Information: Use relevant sections of the 10K report(s), such as MD&A, Financial Statements, Risk Factors, or Notes t

In [14]:
# Create a Langchain chain

from operator import itemgetter

from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import RunnablePassthrough, RunnableSerializable
from langchain_openai import ChatOpenAI

def create_chain (model: str, 
				  prompt_template: ChatPromptTemplate, 
				  retriever: BaseRetriever
				  ) -> RunnableSerializable:

	llm = ChatOpenAI(model=model)
		
	chain = (
		{"context": itemgetter("question") | retriever, "question": itemgetter("question")} 
		| RunnablePassthrough.assign(context=itemgetter("context")) 
		| {"response": prompt_template | llm, "context": itemgetter("context")}
		)

	return chain

#####

def test_create_chain_qdrant():
	docs = process_directory("docs/", "**/*.pdf", PyMuPDFLoader, True)
	chunks = chunk_docs_recursive(docs=docs)

	embeddings = create_embeddings_openai()
	vector_store = create_qdrant_vector_store(":memory:", "holiday-test", 1536, embeddings, chunks)

	retriever = create_retriever_qdrant(vector_store)
	chat_prompt_template = create_chat_prompt_template()
	chain = create_chain("gpt-4o-mini", chat_prompt_template, retriever)
	
	result = chain.invoke({"question" : "What is the annual revenue of Uber?"})
	print(result)

print('\nTESTING QDRANT')
test_create_chain_qdrant()


TESTING QDRANT


100%|██████████| 2/2 [00:00<00:00,  2.21it/s]


{'response': AIMessage(content='The annual revenue of Uber Technologies, Inc. for the year ended December 31, 2023, was $37,281 million (or $37.3 billion). This figure is reported in the Consolidated Statements of Operations section of the 10-K report filed for the period ending December 31, 2023. \n\nFor additional context, the annual revenues for the previous two years were:\n- 2022: $31,877 million\n- 2021: $17,455 million\n\nThis shows a significant increase in revenue year-over-year.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 2083, 'total_tokens': 2198, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-13850aa5-81bd-44af-ac14-

In [15]:
# Generate answers from a chain using a list of questions

import json

from typing import List, Tuple

from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.runnables import RunnableSerializable


def generate_answers_contexts(chain: RunnableSerializable, 
							  questions: list
							  ) -> Tuple[List, List]:
	
	answers = []
	contexts = []

	# Loop over the list of questions and call the chain to get the answer and context
	for question in questions:
		print(question)

		# Call the chain to get answers and contexts
		response = chain.invoke({"question" : question})
		print(response)

		# Capture the answer and context 
		answers.append(response["response"].content)
		contexts.append([context.page_content for context in response["context"]])

	return answers, contexts

#####

def test_generate_answers_contexts():
	docs = process_directory("docs/", "**/*.pdf", PyMuPDFLoader, True)
	chunks = chunk_docs_recursive(docs=docs)

	embeddings = create_embeddings_openai()
	vector_store = create_qdrant_vector_store(":memory:", "holiday-test", 1536, embeddings, chunks)
	
	retriever = create_retriever_qdrant(vector_store)
	chat_prompt_template = create_chat_prompt_template()
	chain = create_chain("gpt-4o-mini", chat_prompt_template, retriever)
	
	questions = ["What is the annual revenue of Lyft?",
			  "What is the annual revenue of Uber?",
			  "Which company has a larger annual revenue - Lyft or Uber?"]	
	
	answers, contexts = generate_answers_contexts(chain=chain, questions=questions)
	
	print(f"Total number of answers = {len(answers)}")
	print(f"Total number of contexts = {len(contexts)}")

test_generate_answers_contexts()

100%|██████████| 2/2 [00:00<00:00,  2.13it/s]


What is the annual revenue of Lyft?
{'response': AIMessage(content='The annual revenue of Lyft for the year ended December 31, 2023, was **$4,403,589,000** (approximately $4.4 billion). This figure is detailed in the Consolidated Statements of Operations in the 10-K report filed on February 20, 2024. \n\nFor reference, the revenue figures for the previous two years were:\n- 2022: **$4,095,135,000**\n- 2021: **$3,208,323,000**\n\nThis information can be found on page 76 of the 10-K report.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 1979, 'total_tokens': 2103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-5a3e98b9-0825-41f2-8055-

In [16]:
# Run a Ragas evaluation 

import time
from typing import Tuple

import pandas as pd
from pandas import DataFrame

from datasets import Dataset
from langchain_core.runnables import RunnableSerializable
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness,
)

def run_ragas_evaluation(chain: RunnableSerializable, 
						questions: list, 
						groundtruths: list, 
						eval_metrics: list = [answer_correctness, answer_relevancy, context_recall, context_precision, faithfulness]
						):
	
	answers = []
	contexts = []
	answers, contexts = generate_answers_contexts(chain=chain, questions=questions)

	# Create the input dataset 
	input_dataset = Dataset.from_dict({"question" : questions,       	# From the dataframe
										"answer" : answers,             # From the chain
										"contexts" : contexts,          # From the chain
										"ground_truth" : groundtruths   # From the dataframe
										})

	# Run the Ragas evaluation using the input dataset and eval metrics
	ragas_results = evaluate(input_dataset, eval_metrics)
	ragas_results_df = ragas_results.to_pandas()
	
	return ragas_results, ragas_results_df
	
#####

def test_run_ragas_evaluation():
	docs = process_directory("docs/", "**/*.pdf", PyMuPDFLoader, True)
	chunks = chunk_docs_recursive(docs=docs)
	
	embeddings = create_embeddings_openai()
	vector_store = create_qdrant_vector_store(":memory:", "holiday-test", 1536, embeddings, chunks)
	
	retriever = create_retriever_qdrant(vector_store)
	chat_prompt_template = create_chat_prompt_template()
	chain = create_chain("gpt-4o-mini", chat_prompt_template, retriever)
	
	testset_df = pd.read_csv("testsets/10k_test_testset.csv")	# This testset contains a small number of questions
	questions = testset_df["question"].values.tolist()
	questions = [str(question) for question in questions]
	groundtruths = testset_df["ground_truth"].values.tolist()
	groundtruths = [str(ground_truth) for ground_truth in groundtruths]  
	eval_metrics = [answer_correctness, answer_relevancy, context_precision, context_recall, faithfulness]
	ragas_results, ragas_results_df = run_ragas_evaluation(chain, questions, groundtruths, eval_metrics)
	
	timestr = time.strftime("%Y%m%d%H%M%S")
	ragas_results_df.to_csv(f"evaluations/10x_test_testset_evaluation_{timestr}.csv")
	
	print(ragas_results)

test_run_ragas_evaluation()

100%|██████████| 2/2 [00:00<00:00,  2.17it/s]


What challenges do jurisdictions pose to business operations in the context of regulatory compliance?
{'response': AIMessage(content="Jurisdictions can pose significant challenges to business operations in the context of regulatory compliance, as highlighted in the 10-K reports for Uber and Lyft. Here are some key points regarding these challenges:\n\n### Uber's 10-K Report:\n1. **Operational Limitations**: Uber may face restrictions in providing or operating its products and services in certain jurisdictions. This can lead to the necessity of modifying its business model in those areas due to local legal requirements (Page 12).\n2. **Regulatory Oversight**: The company is subject to extensive government regulation and oversight concerning payment and financial services, which can impact its operational flexibility and compliance costs (Page 35).\n3. **Legal Risks**: Legal and regulatory risks are prevalent and could adversely affect Uber's ability to consummate transactions, its finan

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

{'answer_correctness': 0.3617, 'answer_relevancy': 0.9113, 'context_precision': 1.0000, 'context_recall': 1.0000, 'faithfulness': 0.7391}


# Test 1 - OpenAI and Qdrant

In [17]:
# Build chain using OpenAI and Qdrant

embeddings = create_embeddings_openai()
docs = process_directory('docs/', '**/*.pdf', PyMuPDFLoader, True)
chunks = chunk_docs_recursive(docs=docs)
print(f'\nNumber of chunks = {len(chunks)}\n')
vector_store = create_qdrant_vector_store(':memory:', 'holiday-test', 1536, embeddings, chunks)
retriever = create_retriever_qdrant(vector_store)
chat_prompt_template = create_chat_prompt_template()
chain = create_chain('gpt-4o', chat_prompt_template, retriever)

100%|██████████| 2/2 [00:00<00:00,  2.23it/s]



Number of chunks = 3398



In [18]:
# Test the chain with a few questions 

questions = ["What is the annual revenue of Uber?",
"What is the annual revenue of Lyft?",
"How does Uber's revenue compare to Lyft's revenue?",]

for question in questions:
	print(question)
	result = chain.invoke({"question" : question})
	print(result)
	print(result["response"].content)
	print("\n*****")

What is the annual revenue of Uber?
{'response': AIMessage(content="The annual revenue of Uber for the years 2021, 2022, and 2023 is as follows:\n\n- **2021:** $17,455 million\n- **2022:** $31,877 million\n- **2023:** $37,281 million\n\nThese figures are extracted from Uber's Consolidated Statements of Operations, as presented in their 10K report for the period ending December 31, 2023.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 2076, 'total_tokens': 2166, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'stop', 'logprobs': None}, id='run-cc38bb9d-d531-4146-896e-eb1125f8b492-0', usage_metadata={'input_tokens': 2076, 'output_tokens': 90, 'total_tokens': 2166}), 'context':

In [20]:
# Evaluate the chain using Ragas

import time

import pandas as pd

# Run the Ragas evaluation and show the results
# Get the questions and groundtruths from the dataframe
testset_df = pd.read_csv("testsets/10k_testset.csv")

questions = testset_df["question"].values.tolist()
questions = [str(question) for question in questions]

groundtruths = testset_df["ground_truth"].values.tolist()
groundtruths = [str(ground_truth) for ground_truth in groundtruths]  

# Specify the eval metrics
eval_metrics = [answer_correctness, answer_relevancy, context_precision, context_recall, faithfulness]

# Run the Ragas evaluation and show the results
ragas_results, ragas_results_df = run_ragas_evaluation(chain, questions, groundtruths, eval_metrics)

# Write the results to disk
timestr = time.strftime("%Y%m%d%H%M%S")
ragas_results_df.to_csv(f"evaluations/10x_test1_testset_evaluation_{timestr}.csv")

# Show the resutls
print(ragas_results)

What challenges do jurisdictions pose to business operations in the context of regulatory compliance?
{'response': AIMessage(content="Jurisdictions present several challenges to business operations regarding regulatory compliance, as highlighted in the 10K reports of companies like Uber and Lyft. Here are the key challenges:\n\n1. **Jurisdictional Restrictions and Business Model Modifications**: Companies may face restrictions or even be blocked from operating in certain jurisdictions. This can require businesses to modify their business models to comply with local regulations, which can be costly and complex. (Referenced from Uber's 10K report, pages 12 and 35)\n\n2. **Extensive Government Regulation**: Businesses are subject to extensive government regulation and oversight, particularly in sectors like payment and financial services. This requires companies to constantly monitor and adapt to a wide array of regulatory requirements, which can vary significantly across jurisdictions. (

Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

{'answer_correctness': 0.5617, 'answer_relevancy': 0.8451, 'context_precision': 0.7083, 'context_recall': 0.9062, 'faithfulness': 0.7834}


# Test 2 - Google Gemini and Vertex AI

## Environment Checks

In [ ]:
# Check user permissions

import os
import sys

import google.auth

# If running in Colab, use the permissions of the currently authenticated user
if 'google.colab' in sys.modules:
	print('Running in Google Colab')
	from google.colab import auth

	auth.authenticate_user()

# If not, set the GOOGLE_APPLICATION_CREDENTIALS to the service account credentials file 
else:
	print('Running locally')
	# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'credentials.json'
	# print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

#####

def test_google_perms():
	from google.cloud import storage
	
	creds, _ = google.auth.default(quota_project_id=os.environ["PROJECT_ID"])

	# Now, you can use the Google Cloud client libraries
	client = storage.Client(credentials=creds)

	# List all buckets in your project
	buckets = list(client.list_buckets())
	print(buckets)

test_google_perms()

In [ ]:
# Check Google API key

import requests

GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
url = f'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={GOOGLE_API_KEY}'
  
headers = {
	"Content-Type": "application/json",
}
  
data = {"contents":[{"parts":[{"text":"What was Uber's annual revenue for 2022?"}]}]}

response = requests.post(url, headers=headers, json=data)
print(response.text)

In [ ]:
# Check Vertex AI Agent Builder data store

import json

import google.auth
from google.auth.transport.requests import Request
import requests

def query_chunks(query: str, 
				page_size: int, 
				access_token:str) -> str:
		
	PROJECT_ID = os.environ['PROJECT_ID']
	LOCATION_ID = os.environ['LOCATION_ID']
	DATA_STORE_ID = os.environ['DATA_STORE_ID']

	if LOCATION_ID == 'us':
		api_endpoint = 'us-discoveryengine.googleapis.com'
	else:
		api_endpoint = 'discoveryengine.googleapis.com'

	url = f"https://{api_endpoint}/v1alpha/projects/{PROJECT_ID}/locations/{LOCATION_ID}/collections/default_collection/dataStores/{DATA_STORE_ID}/servingConfigs/default_search:search"
	print(url)
	
	headers = {
			"Authorization": f"Bearer {access_token}",
			"Content-Type": "application/json",
	}
	
	post_data = {
			"servingConfig": f"projects/{PROJECT_ID}/locations/{LOCATION_ID}/collections/default_collection/dataStores/{DATA_STORE_ID}/servingConfigs/default_search",
			"pageSize": page_size,
			"query": query,
			"contentSearchSpec": {"searchResultMode": "CHUNKS"},
	}
	
	response = requests.post(url, headers=headers, json=post_data)

	if response.status_code != 200:
		print(
				f"Error retrieving search results: {response.status_code} -"
				f" {response.text}"
		)

	return response.json()

#####

def test_query_chunks():
	creds, _ = google.auth.default()
	creds.refresh(Request())
	access_token = creds.token
	print(access_token)
	response = query_chunks(query = "What is the annual revenue of Uber?", page_size=3, access_token=access_token)
	print(json.dumps(response, indent=4))

test_query_chunks()

# Create Synthetic Testset

In [ ]:
# https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/
# I'm using an older version of Ragas because I couldn't the current version to work 

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from ragas.testset.evolutions import multi_context, reasoning, simple
from ragas.testset.generator import RunConfig, TestsetGenerator

# Split the documents into chunks
docs = process_directory('docs/', '**/*.pdf', PyMuPDFLoader, True)
chunks = chunk_docs_recursive(docs=docs, chunk_size=1000, chunk_overlap=200)

generator_llm = ChatOpenAI(model="gpt-4o-mini")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
embeddings = OpenAIEmbeddings()

# Create the generator 
generator = TestsetGenerator.from_langchain(generator_llm=generator_llm, critic_llm=critic_llm, embeddings=embeddings)

# https://docs.ragas.io/en/v0.1.21/howtos/customisations/run_config.html
# Default is 16 but using a smaller number is used to avoid rate limits
run_config=RunConfig(max_workers=8)

# Set the number of questions
test_size=30

# Set the distribution 
distributions = {simple: 0.2, multi_context: 0.4, reasoning: 0.4}

# Generate the testset and save to disk 
testset = generator.generate_with_langchain_docs(documents=docs, test_size=test_size, distributions=distributions, run_config=run_config)

# Write the testet to disk
testset_name = "10k_testset.csv"
print(f"Saving {testset_name}")
testset_df = testset.to_pandas()
testset_df.to_csv(f"{testset_name}")
testset_df